In [2]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [3]:
# read in with pandas
winter1 = pd.read_csv ('jpx20-21data.txt')
winter2 = pd.read_csv ('jpx21-22data.txt')
winter3 = pd.read_csv ('jpx22-23data.txt')

In [4]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [5]:
# mask out days without precipitation
codes = ['GR','GS','IC','PL','PY','RA','SG','SH','SN','SQ','TS','UP']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [6]:
precip_days2

,station,datetime,precip(mm),wxcodes
93,JPX,2021-12-02 07:15:00,0.00,RA
105,JPX,2021-12-02 11:15:00,0.00,-RA
156,JPX,2021-12-03 04:15:00,0.00,-RA
383,JPX,2021-12-06 07:55:00,0.00,-RA
386,JPX,2021-12-06 08:55:00,0.00,TS
...,...,...,...,...
6195,JPX,2022-02-25 14:55:00,6.60,RA
6196,JPX,2022-02-25 15:15:00,1.27,-RA
6197,JPX,2022-02-25 15:35:00,5.08,RA
6198,JPX,2022-02-25 15:55:00,5.33,-RA


In [7]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_22052\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_22052\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_22052\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [8]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_22052\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_22052\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_22052\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [9]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 55]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 55]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 55]

In [10]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [11]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [12]:
jpx = LocationInfo(
    'East Hampton', 'New York', 
    latitude=40.9634, 
    longitude=-72.1848, 
    timezone='America/New_York'
)

jpx

LocationInfo(name='East Hampton', region='New York', timezone='America/New_York', latitude=40.9634, longitude=-72.1848)

In [13]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(jpx.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [14]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(jpx.observer, date=date)
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [15]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(jpx.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [16]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 06:54:54  2020-12-01 16:20:53  2020-12-01
1   2020-12-02 06:55:55  2020-12-02 16:20:40  2020-12-02
2   2020-12-03 06:56:54  2020-12-03 16:20:29  2020-12-03
3   2020-12-04 06:57:53  2020-12-04 16:20:20  2020-12-04
4   2020-12-05 06:58:50  2020-12-05 16:20:13  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:30:15  2021-02-24 17:34:05  2021-02-24
86  2021-02-25 06:28:46  2021-02-25 17:35:15  2021-02-25
87  2021-02-26 06:27:16  2021-02-26 17:36:25  2021-02-26
88  2021-02-27 06:25:45  2021-02-27 17:37:34  2021-02-27
89  2021-02-28 06:24:14  2021-02-28 17:38:44  2021-02-28

[90 rows x 3 columns]


In [17]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 06:54:39  2021-12-01 16:20:56  2021-12-01
1   2021-12-02 06:55:40  2021-12-02 16:20:42  2021-12-02
2   2021-12-03 06:56:40  2021-12-03 16:20:31  2021-12-03
3   2021-12-04 06:57:39  2021-12-04 16:20:21  2021-12-04
4   2021-12-05 06:58:36  2021-12-05 16:20:14  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:30:37  2022-02-24 17:33:48  2022-02-24
86  2022-02-25 06:29:08  2022-02-25 17:34:58  2022-02-25
87  2022-02-26 06:27:38  2022-02-26 17:36:08  2022-02-26
88  2022-02-27 06:26:07  2022-02-27 17:37:17  2022-02-27
89  2022-02-28 06:24:36  2022-02-28 17:38:27  2022-02-28

[90 rows x 3 columns]


In [18]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 06:54:24  2022-12-01 16:21:00  2022-12-01
1   2022-12-02 06:55:26  2022-12-02 16:20:45  2022-12-02
2   2022-12-03 06:56:26  2022-12-03 16:20:33  2022-12-03
3   2022-12-04 06:57:25  2022-12-04 16:20:23  2022-12-04
4   2022-12-05 06:58:22  2022-12-05 16:20:16  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:30:58  2023-02-24 17:33:30  2023-02-24
86  2023-02-25 06:29:29  2023-02-25 17:34:41  2023-02-25
87  2023-02-26 06:28:00  2023-02-26 17:35:51  2023-02-26
88  2023-02-27 06:26:29  2023-02-27 17:37:00  2023-02-27
89  2023-02-28 06:24:58  2023-02-28 17:38:10  2023-02-28

[90 rows x 3 columns]


In [19]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

C:\Users\alyha\AppData\Local\Temp\ipykernel_22052\762931898.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['date'] = precip_days2['datetime'].dt.date


In [20]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,JPX,2020-12-04 20:55:00,0.00,-RA,2020-12-04,2020-12-04 06:57:53,2020-12-04 16:20:20
1,JPX,2020-12-05 05:55:00,0.00,-RA,2020-12-05,2020-12-05 06:58:50,2020-12-05 16:20:13
2,JPX,2020-12-05 06:55:00,0.25,-RA,2020-12-05,2020-12-05 06:58:50,2020-12-05 16:20:13
3,JPX,2020-12-05 08:55:00,0.51,-RA,2020-12-05,2020-12-05 06:58:50,2020-12-05 16:20:13
4,JPX,2020-12-05 09:55:00,2.03,RA,2020-12-05,2020-12-05 06:58:50,2020-12-05 16:20:13
...,...,...,...,...,...,...,...
193,JPX,2021-02-23 19:55:00,0.00,-RA,2021-02-23,2021-02-23 06:31:43,2021-02-23 17:32:54
194,JPX,2021-02-27 13:55:00,0.00,RA,2021-02-27,2021-02-27 06:25:45,2021-02-27 17:37:34
195,JPX,2021-02-27 14:55:00,1.02,-RA,2021-02-27,2021-02-27 06:25:45,2021-02-27 17:37:34
196,JPX,2021-02-27 15:55:00,1.78,-RA,2021-02-27,2021-02-27 06:25:45,2021-02-27 17:37:34


In [21]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,JPX,2021-12-06 07:55:00,0.00,-RA,2021-12-06,2021-12-06 06:59:32,2021-12-06 16:20:10
1,JPX,2021-12-06 08:55:00,0.00,TS,2021-12-06,2021-12-06 06:59:32,2021-12-06 16:20:10
2,JPX,2021-12-08 15:55:00,0.00,-RA,2021-12-08,2021-12-08 07:01:21,2021-12-08 16:20:07
3,JPX,2021-12-08 16:55:00,0.25,-RA,2021-12-08,2021-12-08 07:01:21,2021-12-08 16:20:07
4,JPX,2021-12-08 17:55:00,1.78,-RA,2021-12-08,2021-12-08 07:01:21,2021-12-08 16:20:07
...,...,...,...,...,...,...,...
201,JPX,2022-02-25 11:55:00,2.03,-RA,2022-02-25,2022-02-25 06:29:08,2022-02-25 17:34:58
202,JPX,2022-02-25 12:55:00,1.02,-RA,2022-02-25,2022-02-25 06:29:08,2022-02-25 17:34:58
203,JPX,2022-02-25 13:55:00,5.08,-RA,2022-02-25,2022-02-25 06:29:08,2022-02-25 17:34:58
204,JPX,2022-02-25 14:55:00,6.60,RA,2022-02-25,2022-02-25 06:29:08,2022-02-25 17:34:58


In [22]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,JPX,2022-12-01 00:55:00,0.76,-RA,2022-12-01,2022-12-01 06:54:24,2022-12-01 16:21:00
1,JPX,2022-12-03 15:55:00,0.00,RA,2022-12-03,2022-12-03 06:56:26,2022-12-03 16:20:33
2,JPX,2022-12-03 16:55:00,0.51,RA,2022-12-03,2022-12-03 06:56:26,2022-12-03 16:20:33
3,JPX,2022-12-03 17:55:00,1.52,-RA,2022-12-03,2022-12-03 06:56:26,2022-12-03 16:20:33
4,JPX,2022-12-03 18:55:00,2.79,RA,2022-12-03,2022-12-03 06:56:26,2022-12-03 16:20:33
...,...,...,...,...,...,...,...
163,JPX,2023-02-22 22:55:00,0.25,-RA,2023-02-22,2023-02-22 06:33:53,2023-02-22 17:31:09
164,JPX,2023-02-22 23:55:00,0.25,-RA,2023-02-22,2023-02-22 06:33:53,2023-02-22 17:31:09
165,JPX,2023-02-25 16:55:00,0.00,-SN,2023-02-25,2023-02-25 06:29:29,2023-02-25 17:34:41
166,JPX,2023-02-25 17:55:00,0.00,-SN,2023-02-25,2023-02-25 06:29:29,2023-02-25 17:34:41


In [23]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
3,JPX,2020-12-05 08:55:00,0.51,-RA,2020-12-05,2020-12-05 06:58:50,2020-12-05 16:20:13
4,JPX,2020-12-05 09:55:00,2.03,RA,2020-12-05,2020-12-05 06:58:50,2020-12-05 16:20:13
5,JPX,2020-12-05 10:55:00,3.81,-RA,2020-12-05,2020-12-05 06:58:50,2020-12-05 16:20:13
6,JPX,2020-12-05 11:55:00,0.51,-RA,2020-12-05,2020-12-05 06:58:50,2020-12-05 16:20:13
7,JPX,2020-12-05 12:55:00,6.60,RA,2020-12-05,2020-12-05 06:58:50,2020-12-05 16:20:13
...,...,...,...,...,...,...,...
178,JPX,2021-02-19 13:55:00,0.51,-SN,2021-02-19,2021-02-19 06:37:27,2021-02-19 17:28:09
179,JPX,2021-02-19 16:55:00,0.25,UP,2021-02-19,2021-02-19 06:37:27,2021-02-19 17:28:09
194,JPX,2021-02-27 13:55:00,0.00,RA,2021-02-27,2021-02-27 06:25:45,2021-02-27 17:37:34
195,JPX,2021-02-27 14:55:00,1.02,-RA,2021-02-27,2021-02-27 06:25:45,2021-02-27 17:37:34


In [24]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_22052\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_22052\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_22052\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [25]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
3,JPX,2020-12-05 08:55:00,0.51,-RA,2020-12-05,2020-12-05 06:58:50,2020-12-05 16:20:13,2020-12-05 07:58:50,2020-12-05 15:20:13
4,JPX,2020-12-05 09:55:00,2.03,RA,2020-12-05,2020-12-05 06:58:50,2020-12-05 16:20:13,2020-12-05 07:58:50,2020-12-05 15:20:13
5,JPX,2020-12-05 10:55:00,3.81,-RA,2020-12-05,2020-12-05 06:58:50,2020-12-05 16:20:13,2020-12-05 07:58:50,2020-12-05 15:20:13
6,JPX,2020-12-05 11:55:00,0.51,-RA,2020-12-05,2020-12-05 06:58:50,2020-12-05 16:20:13,2020-12-05 07:58:50,2020-12-05 15:20:13
7,JPX,2020-12-05 12:55:00,6.60,RA,2020-12-05,2020-12-05 06:58:50,2020-12-05 16:20:13,2020-12-05 07:58:50,2020-12-05 15:20:13
...,...,...,...,...,...,...,...,...,...
177,JPX,2021-02-19 12:55:00,0.00,-SN,2021-02-19,2021-02-19 06:37:27,2021-02-19 17:28:09,2021-02-19 07:37:27,2021-02-19 16:28:09
178,JPX,2021-02-19 13:55:00,0.51,-SN,2021-02-19,2021-02-19 06:37:27,2021-02-19 17:28:09,2021-02-19 07:37:27,2021-02-19 16:28:09
194,JPX,2021-02-27 13:55:00,0.00,RA,2021-02-27,2021-02-27 06:25:45,2021-02-27 17:37:34,2021-02-27 07:25:45,2021-02-27 16:37:34
195,JPX,2021-02-27 14:55:00,1.02,-RA,2021-02-27,2021-02-27 06:25:45,2021-02-27 17:37:34,2021-02-27 07:25:45,2021-02-27 16:37:34


In [26]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [27]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3


In [28]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 1.8715789473684212
2021-2022: 1.3802631578947366
2022-2023: 0.9609523809523809


In [29]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,JPX,2020-12-04 20:55:00,0.00,-RA,2020-12-04,2020-12-04 06:57:53,2020-12-04 16:20:20
1,JPX,2020-12-05 05:55:00,0.00,-RA,2020-12-05,2020-12-05 06:58:50,2020-12-05 16:20:13
2,JPX,2020-12-05 06:55:00,0.25,-RA,2020-12-05,2020-12-05 06:58:50,2020-12-05 16:20:13
11,JPX,2020-12-05 16:55:00,3.56,RA,2020-12-05,2020-12-05 06:58:50,2020-12-05 16:20:13
12,JPX,2020-12-05 17:55:00,2.03,-RA,2020-12-05,2020-12-05 06:58:50,2020-12-05 16:20:13
...,...,...,...,...,...,...,...
190,JPX,2021-02-22 20:55:00,1.78,-RA,2021-02-22,2021-02-22 06:33:11,2021-02-22 17:31:43
191,JPX,2021-02-22 21:55:00,2.79,RA,2021-02-22,2021-02-22 06:33:11,2021-02-22 17:31:43
192,JPX,2021-02-22 22:55:00,3.05,-RA,2021-02-22,2021-02-22 06:33:11,2021-02-22 17:31:43
193,JPX,2021-02-23 19:55:00,0.00,-RA,2021-02-23,2021-02-23 06:31:43,2021-02-23 17:32:54


In [30]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_22052\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_22052\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_22052\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [31]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [32]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [33]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.7854716981132075
2021-2022: 0.914
2022-2023: 0.8823913043478262
